In [1]:
# Q1. take a data set from attached URL 
# https://archive.ics.uci.edu/ml/datasets/Car+Evaluation
# Q2. create a database called as cardataset

# Q3. create a table called as car with a column name given in data 
# description

# Q4. Dump all the data into car table
# Q5.try to check wether all the data is available inside your table or not

# Q6. try to group all the data with col1 and count occurences of each and 
# every record based on col1 value

#Q7. try to filter a record where col3 value will be 4
# Q8. try to update a col3 value with 8 wherever you have value equal to 2
# Q9. try to delete table
# Q10. try to delete database

In [2]:

import sys

In [3]:
sys.path.insert(1,'C:\\Users\\SREEMAN\\iNeuron\\Assignments\\home-work\\homw-work-3_feb_07_2022 (mysql)\\my_sql_pkg')

In [4]:
import logger

In [5]:
log = logger.log()

In [6]:
import mysql.connector as conn
class Database:
    __COLUMNS_REQ = 7
    def __init__ (self):
        try:
            self.__mydb = conn.connect(host = "localhost", user = "root", password = "mysql")
            self.__cursor = self.__mydb.cursor()
        except:
            log.error("Server could not connect! Some error occured")
        else:
            log.info("Server Connected")
            
    def create_db (self,db_name = "cardataset"):
        try:
            self.__cursor.execute("create database {db}".format(db = db_name))
        except Exception as e:
            log.warning('{e_val}'.format(e_val=e))
    
    def show_db (self):
        self.__cursor.execute("show databases")
        return self.__cursor.fetchall()
    
    def drop_db (self,db_name):
        self.__cursor.execute("drop database if exists {db_var}".format(db_var = db_name))
        self.__mydb.commit()
    
    def __check_db (self,db_name):
        try:
            # check if db exists & use it
            self.__cursor.execute("use {db_val}".format(db_val = db_name))
        except Exception as e:
            log.error("DataBase not found!",e)
            return True
        else:
            log.info("DataBase present")
            return False
            
    def __check_table(self,table_name):
        try:
            self.__cursor.execute("select * from {table} where 1".format(table = table_name))
            self.__cursor.fetchall()
        except Exception as e:
            log.warning("New Table",e)
            return True
        else:
            log.warning("Table {table} already exists".format(table=table_name))
            return False

    def create_table(self,table_name,columns = (),db_name = "cardataset"):
        
        # ERROR_1 -> check if database is present
        if self.__check_db(db_name):
            return
        # create the table
        # ERROR_2 -> check if table exists:
        if self.__check_table(table_name):
            if len(columns) == self.__COLUMNS_REQ:
                try:
                    self.__cursor.execute("create table {table_}({col1},{col2},{col3},{col4},{col5},"
                                        "{col6},{col7})".format(table_=table_name,col1=columns[0],
                                                         col2=columns[1],col3=columns[2],
                                                         col4=columns[3],col5=columns[4],
                                                         col6=columns[5],col7=columns[6]))
                except Exception as e:
                    log.error("Could not create the table!",e)
                else:
                    log.info("Table created successfully")
    
    def show_table(self,db_name):
        # Error_1 -> check db_name
        if self.__check_db(db_name):
            return
        self.__cursor.execute("use {db_}".format(db_ = db_name))
        self.__cursor.execute("show tables")
        return self.__cursor.fetchall()
    
    def describe_table (self,db_name,table_name):
        # Error_1 -> check db_name
        if self.__check_db(db_name) or self.__check_table(table_name):
            return
        self.__cursor.execute("use {db_}".format(db_ = db_name))
        self.__cursor.execute("describe {table}".format(table = table_name))
        return self.__cursor.fetchall()
    
    def drop_table (self,table_name,db_name):
        self.__cursor.execute("drop table if exists {db_}.{table}".format(db_ = db_name,table = table_name))
        self.__mydb.commit()
    
    def __check_val_str(self,insert_str):
        delimiter = ","
        val_lst = insert_str.split(delimiter)
        if (len(val_lst) == self.__COLUMNS_REQ):
            return True
        else:
            log.error("Insufficient values for columns")
            return False
        
    def __single_insert (table_name,single_insert):
        if not self.__check_val_str(single_insert):
            return
        self.__cursor.execute("INSERT INTO {table} VALUES ({val})".format(table=table_name,
                                                                      val = single_insert))
        self.__mydb.commit()
    
    def __file_insert (table_name,file_insert):
        import csv
        try:
            with open(file_insert,"r") as fhand:
                data = csv.reader(fhand, delimiter = "\n")
                for i in data:
                    var_lst = (i[0].split(","))
                    str_val = ""
                    # convert the list of values to sql query string
                    for val in var_lst:
                        str_val = str_val + "'" + val + "'" + ", "
                    str_val = str_val[:-2]
                    # insert the query into table
                    self.__cursor.execute("insert into {table} VALUES ({val})".format(
                            table=table_name, val = str_val))
                self.__mydb.commit()
        except Exception as e:
            log.error(e)
    
    def insert (self,db_name,table_name,single_insert = None,file_insert = None):
        # Error_1 -> check db_name and ERROR_2 -> check if table exists:
        if self.__check_db(db_name) or self.__check_table(table_name):
            return
        
        if (single_insert is not None) and (type(single_insert)==str):
            self.__single_insert(table_name,single_insert)
        
        elif (file_insert is not None):
            self.__file_insert(table_name,file_insert)
                    
    def select (self,db_name="cardataset",table_name="car",cols = "*"):
        # ERROR_1 -> check if database is present
        exp_db = self.__check_db(db_name)
        if exp_db:
            return 
        self.__cursor.execute("select {col_val} from {table_val}".format(col_val = cols,
                                                            table_val = table_name))
        return self.__cursor.fetchall()
    
    def count_records (self,db_name = "cardataset",table_name = "car",cols="*",
                      nest_query=False):
        # Error_1 -> check db_name and ERROR_2 -> check if table exists:
        if self.__check_db(db_name) or self.__check_table(table_name):
            return
        self.__cursor.execute("select count({cols_var}) from {table}".format(cols_var = cols,
                                                            table = table_name))
        return self.__cursor.fetchall()
            
            
    def group_by (self,db_name = "cardataset",table_name="car",gp_col = "buying"):
        # Error_1 -> check db_name and ERROR_2 -> check if table exists:
        if self.__check_db(db_name) or self.__check_table(table_name):
            return
        # group_by col 
        self.__cursor.execute("select {gp_col_var},count(*) as count_records from {table} "
                    "group by {gp_col_var}".format(table = table_name,gp_col_var = gp_col))
        return self.__cursor.fetchall()
    
    def filter_rec (self,filter_val,db_name = "cardataset", table_name = "car",
                    filter_col = "doors",cols = "*"):
        # Error_1 -> check db_name and ERROR_2 -> check if table exists:
        if self.__check_db(db_name) or self.__check_table(table_name):
            return
        self.__cursor.execute("select {cols_var} from {table} where {filt_col} = {filt_val}".
                              format(cols_var = cols,table=table_name,filt_col=filter_col,
                              filt_val = filter_val))
        return self.__cursor.fetchall()
    
    def update (self,db_name = "cardataset", table_name = "car",update_col = "doors",
                prev_val = "2", upd_val = "8"):
        # Error_1 -> check db_name and ERROR_2 -> check if table exists:
        if self.__check_db(db_name) or self.__check_table(table_name):
            return
        self.__cursor.execute("update {table} set {upd_col} = {upd_val_var} where {upd_col} like "
                              "{prev_val_var}".format(table=table_name,upd_col=update_col,
                                upd_val_var = upd_val,prev_val_var = prev_val))
        self.__mydb.commit()

In [7]:
db_ = "cardataset"
table = "car"

In [8]:
db = Database()

In [9]:
db.show_db()

[('cardataset',),
 ('ineuron',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',),
 ('world',)]

In [81]:
db.group_by()

DataBase present
Table car already exists


[('vhigh', 435), ('high', 432), ('med', 432), ('low', 432)]

In [82]:
db.count_records()

DataBase present
Table car already exists


[(1731,)]

In [92]:
db.filter_rec(4)

DataBase present
Table car already exists


[('vhigh', 'vhigh', '4', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'small', 'med', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'small', 'high', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'med', 'low', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'med', 'med', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'med', 'high', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'big', 'low', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'big', 'med', 'unacc'),
 ('vhigh', 'vhigh', '4', '2', 'big', 'high', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'small', 'med', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'small', 'high', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'med', 'low', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'med', 'med', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'med', 'high', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'big', 'low', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'big', 'med', 'unacc'),
 ('vhigh', 'vhigh', '4', '4', 'big', 'high', 'unacc'),
 ('vhigh',

In [4]:
db.update()

DataBase present
Table car already exists


In [8]:
db.filter_rec(8)

DataBase present
Table car already exists


[('vhigh', 'vhigh', '8', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'small', 'med', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'small', 'high', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'med', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'med', 'med', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'med', 'high', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'big', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'big', 'med', 'unacc'),
 ('vhigh', 'vhigh', '8', '2', 'big', 'high', 'unacc'),
 ('vhigh', 'vhigh', '8', '4', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '4', 'small', 'med', 'unacc'),
 ('vhigh', 'vhigh', '8', '4', 'small', 'high', 'unacc'),
 ('vhigh', 'vhigh', '8', '4', 'med', 'low', 'unacc'),
 ('vhigh', 'vhigh', '8', '4', 'med', 'med', 'unacc'),
 ('vhigh', 'vhigh', '8', '4', 'med', 'high', 'unacc'),
 ('vh

In [9]:
db.select("cardataset","car",cols = "buying,maint")

DataBase present


[('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'vhigh'),
 ('vhigh', 'v

In [210]:
db.insert("cardataset","car",file_insert = "car.data")

DataBase present
Table car already exists


In [211]:
db.select("cardataset","car")

DataBase present


[('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'med', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'high', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'med', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'med', 'med', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'med', 'high', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'big', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'big', 'med', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'big', 'high', 'unacc'),
 ('vhigh', 'vhigh', '2', '4', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '4', 'small', 'med', 'unacc'),
 ('vhigh', 'vhigh', '2', '4', 'small', 'high', 'unacc'),
 ('vhigh', 'vhigh', '2', '4', 'med', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '4', 'med', 'med', 'unacc'),
 ('vhigh', 'vhigh', '2', '4', 'med', 'high', 'unacc'),
 ('vh

In [196]:
import csv
with open("car.data","r") as fhand:
    car_data = csv.reader(fhand, delimiter = "\n")
    for i in car_data:
        var_lst = (i[0].split(","))
        str_val = ""
        for val in var_lst:
            str_val = str_val + "'" + val + "'" + ", "
        str_val = str_val[:-2]
        print(str_val)
        break

'vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'


In [151]:
db.insert("cardataset","car",single_insert="'vhigh','vhigh','2','2','small','low','unacc'")

DataBase present
Table car already exists


In [152]:
db.cursor.execute("select * from cardataset.car")

In [153]:
db.cursor.fetchall()

[('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc'),
 ('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc')]

In [124]:
tup = ("vhigh,vhigh,2,2,small,low,unacc",)

In [130]:
var_lst = tup[0].split(",")
print(var_lst)

['vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc']


In [134]:
(tuple(var_lst))

('vhigh', 'vhigh', '2', '2', 'small', 'low', 'unacc')

In [73]:
#db.__cursor.execute("insert into {db_val}.{table} values ({val})".format(db_val = "cardataset",
 #                                       table = "car", val = var_lst))

In [79]:
db.cursor.fetchall()

[]

In [80]:
db.show_db()

[('cardataset',),
 ('ineuron',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',),
 ('world',)]

In [81]:
db.cursor.fetchall()

[]

In [ ]:
db.drop_table("car","cardataset")

In [82]:
db.show_table("cardataset")

DataBase present


[('car',)]

In [69]:
db.describe_table("cardataset","car")

DataBase present
Table car already exists


[('buying', b'varchar(10)', 'YES', '', None, ''),
 ('maint', b'varchar(10)', 'YES', '', None, ''),
 ('doors', b'varchar(10)', 'YES', '', None, ''),
 ('persons', b'varchar(10)', 'YES', '', None, ''),
 ('lug_boot', b'varchar(10)', 'YES', '', None, ''),
 ('safety', b'varchar(10)', 'YES', '', None, ''),
 ('Class_Distribution', b'varchar(10)', 'YES', '', None, '')]

In [83]:
columns_ = ("buying varchar(10)","maint varchar(10)","doors varchar(10)",
           "persons varchar(10)","lug_boot varchar(10)","safety varchar(10)",
           "Class_Distribution varchar(10)")

In [84]:
db.create_table("car",columns = columns_,db_name = "cardataset")

DataBase present
Table car already exists


In [85]:
db.show_table("cardataset")

DataBase present


[('car',)]

In [95]:
db.cursor.fetchall()

[]

In [116]:
db.cursor.execute("select * from cardataset.car")

In [117]:
db.cursor.fetchall()

[]